# Graph of US counties and census tracts

This notebook creates a graph based on US counties and census tract administrative units.

In order to create the graph, we use [the Tiger shapefiles](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341).

In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx

In [2]:
def create_graph(tracts, ix="GEOID", lon="INTPTLON", lat="INTPTLAT"):
    geom = tracts.set_index(ix)

    L = tracts[[ix, lat, lon]].set_index(ix)
    L[lat] = L[lat].astype("float")
    L[lon] = L[lon].astype("float")

    feats = [c for c in L.columns if c != "geometry"]

    g = nx.Graph()
    g.add_nodes_from(geom.index.values)

    # make edge list from GeoPandas DataFrame
    edge_list = []
    for index, row in geom.iterrows():
        for f in feats:
            g.nodes[index][f] = L.loc[index][f]
        nbrs = geom[geom.geometry.touches(row.geometry)].index.values
        for nbr in nbrs:
            edge_list.append((index, nbr))
    g.add_edges_from(edge_list)
    return g

## CA census tract graph

In [3]:
g = create_graph(gpd.read_file("shapefiles/tl_2020_06_tract/tl_2020_06_tract.shp"))
nx.write_graphml(g, "../data/ca_tracts.graphml")

## TX census tract graph

In [4]:
g = create_graph(gpd.read_file("shapefiles/tl_2020_48_tract/tl_2020_48_tract.shp"))
nx.write_graphml(g, "../data/tx_tracts.graphml")

## Counties

In [5]:
counties = gpd.read_file("shapefiles/tl_2010_us_county10/tl_2010_us_county10.shp")
counties = counties[~counties.STATEFP10.isin(["02", "72", "78", "15"])]

In [6]:
g = create_graph(counties, ix="GEOID10", lon="INTPTLON10", lat="INTPTLAT10")
nx.write_graphml(g, "../data/counties.graphml")